In [21]:
import torch
import numpy as np


def get_division_tree(n_agents):
    agent2divitreeindex = np.arange(n_agents)
    np.random.shuffle(agent2divitreeindex)
    max_div = np.ceil(np.log2(n_agents)).astype(int)
    levels = np.zeros(shape=(max_div+1, n_agents), dtype=int)
    tree_of_agent = []*(max_div+1)
    for ith, level in enumerate(levels):
        if ith == 0: continue
        levels[ith,:]  = levels[ith-1,:]
        for j in range(n_agents):
            seg = j // ( n_agents /2**ith)
            if seg%2==1:
                levels[ith,j] += 2**(ith-1)
    res_levels = levels.copy()
    for i, div_tree_index in enumerate(agent2divitreeindex):
        res_levels[:, i] = levels[:, div_tree_index]
    return res_levels


res = get_division_tree(15)
for r in res:
    print(r)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 1 1 0 1 1 0 1 1]
[3 2 0 0 0 0 2 1 1 2 3 1 2 3 1]
[7 2 0 0 4 4 2 1 5 6 3 5 6 3 1]
[ 7 10  8  0  4 12  2  1  5 14 11 13  6  3  9]


In [3]:
agent2divitreeindex = np.arange(n_agents)

NameError: name 'n_agents' is not defined

In [139]:
import torch
import numpy as np
from torch.distributions.categorical import Categorical

def _get_act_log_probs(distribution, action):
    return distribution.log_prob(action.squeeze(-1)).unsqueeze(-1)

res = []
for i in range(10000):

    hyper_act_logits = torch.Tensor([0.5, 0.5])
    logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
    test_mode = False
    eval_mode = False

    h_dist = Categorical(logits = hyper_act_logits)
    # print('h_dist:', h_dist.probs)
    if not test_mode:  h_act = h_dist.sample() if not eval_mode else eval_actions
    else:              h_act = torch.argmax(h_dist.probs)
    # print('h_act(1为采样, 0为最大):', h_act)

    hActLogProbs = _get_act_log_probs(h_dist, h_act)
    # print('hActProbs:', torch.exp(hActLogProbs))


    act_dist = Categorical(logits = logits_agent_cluster)
    # print('act_dist:', act_dist.probs)
    act_sample = act_dist.sample() if not eval_mode else eval_actions
    act_argmax = torch.argmax(act_dist.probs)

    act = torch.where(h_act==1, act_sample, act_argmax) # h_act: shape=($n_thread, $n_agent)
    sel_argmax = (act_argmax==act)
    # print('act:', act)

    actLogProbs01 = _get_act_log_probs(act_dist, act) # the policy gradient loss will feedback from here
    # print('actProbs01:', torch.exp(actLogProbs01))


    actLogProbs_notargmax = actLogProbs01 + hActLogProbs
    actLogProbs_argmax = torch.log(torch.exp(actLogProbs01 + hActLogProbs) + (1-torch.exp(hActLogProbs)))
    actLogProbs = torch.where(sel_argmax, actLogProbs_argmax, actLogProbs_notargmax)
    # print('actProbs:', torch.exp(actLogProbs))
    res.append(act.item())
res = np.array(res)
len_res = len(res)
print(sum(res==0)/len_res)
print(sum(res==1)/len_res)
print(sum(res==2)/len_res)

0.7296
0.1399
0.1305


client.send_targeted_dgram('ddd')

In [94]:
logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
# torch.nn.functional.gumbel_softmax(logits, tau=1, hard=False, eps=1e-10, dim=- 1)
q = {0:0,1:0,2:0}
for i in range(10000):
    torch.set_printoptions(precision=3, sci_mode=False)
    t = torch.nn.functional.gumbel_softmax(logits_agent_cluster, tau=0.2).argmax().item()
    q[t] += 1

In [95]:
q

{0: 4670, 1: 2757, 2: 2573}

In [101]:
torch.softmax(torch.Tensor([0.7, 0.2, 0.1]), -1)

tensor([0.464, 0.281, 0.255])

In [ ]:


def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    U = U.cuda()
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature=1):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature=1, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    
    if not hard:
        return y

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard